In [22]:
# Read from Hop's Github to pull bridge tokens and addresses for Dune's spell table
# Right now, we manually run this and copy and paste to Dune's spellbook
# it would be cool to automate this process some day...

import requests
import re
import json
import pandas as pd

gh_api = 'https://api.github.com/repos/OWNER/REPO/contents/PATH'
#sourcefile = 'https://github.com/hop-protocol/hop/blob/develop/packages/core/src/addresses/mainnet.ts'
owner_api = 'hop-protocol'
repo_api='hop'
path_api = 'packages/core/src/addresses/'
file_names = ['mainnet.ts','types.ts']

In [23]:
for a_fn in file_names:
        rep = {'OWNER': owner_api, 'REPO': repo_api, 'PATH':path_api + a_fn} # define desired replacements here
        ## use these three lines to do the replacement
        rep = dict((re.escape(k), v) for k, v in rep.items()) 
        ## Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
        rep_pattern = re.compile("|".join(rep.keys()))
        req_api = rep_pattern.sub(lambda m: rep[re.escape(m.group(0))], gh_api)
        # print(req_api)

        inf = requests.get(req_api)
        dl_url = inf.json()['download_url']
        print(dl_url)

        r = requests.get(dl_url)
        f = open(a_fn, 'wb')
        for chunk in r.iter_content(chunk_size=512 * 1024): 
                if chunk:
                        f.write(chunk)
        f.close()


# json = json.dumps(content)
# print(json)

https://raw.githubusercontent.com/hop-protocol/hop/develop/packages/core/src/addresses/mainnet.ts
https://raw.githubusercontent.com/hop-protocol/hop/develop/packages/core/src/addresses/types.ts


In [24]:
# read jsons to intermediates folder
! node hop_transpile.js

In [25]:
bridges = pd.read_json("intermediates/hop_bridges.txt")
bridges.reset_index(inplace=True)

bridges_unpivot = pd.melt(bridges, id_vars='index')
bridges_unpivot = bridges_unpivot[~bridges_unpivot['value'].isna()]

# expand data in json to columns
json_data = bridges_unpivot['value'].values
normalized_df = pd.json_normalize(json_data)
bridges = bridges_unpivot.join(normalized_df)
# drop original json column
bridges = bridges.drop('value', axis=1)
#rename columns
bridges = bridges.rename(columns={
        'index':'blockchain',
        'variable':'token_symbol'
})


int_cols = ['bridgeDeployedBlockNumber']

for col in bridges.columns:
        if col in int_cols:
                bridges[col] = bridges[col].fillna(-1)
                bridges[col] = bridges[col].astype('int64')
                bridges[col] = ',' + bridges[col].astype(str)
        else:
                bridges[col] = bridges[col].fillna('')
                bridges[col] = ',\'' + bridges[col].astype(str) + '\''

# Add parenthases for SQL
bridges.iloc[:, 0] = '(' + bridges.iloc[:, 0].str.replace(',','') #replace leading comma
# Add comma for all except the first row
bridges.iloc[1:, 0] = ',' + bridges.iloc[1:, 0].astype(str)
# add parenthases to last row
bridges.iloc[:, -1] = bridges.iloc[:, -1].astype(str) + ')'
# bridges['blockchain'] = ',(' + bridges['blockchain'].astype(str)


col_names = ['`' + c + '`' for c in bridges.columns]
col_names = ",".join(col_names)
print(col_names)
display(bridges)



`blockchain`,`token_symbol`,`l1CanonicalToken`,`l1Bridge`,`bridgeDeployedBlockNumber`,`l1CanonicalBridge`,`l1MessengerWrapper`,`l2CanonicalBridge`,`l2CanonicalToken`,`l2Bridge`,`l2HopBridgeToken`,`l2AmmWrapper`,`l2SaddleSwap`,`l2SaddleLpToken`,`l1Amb`,`l2Amb`,`l1FxBaseRootTunnel`,`l1PosRootChainManager`,`l1PosPredicate`,`l2MessengerProxy`


,blockchain,token_symbol,l1CanonicalToken,l1Bridge,bridgeDeployedBlockNumber,l1CanonicalBridge,l1MessengerWrapper,l2CanonicalBridge,l2CanonicalToken,l2Bridge,l2HopBridgeToken,l2AmmWrapper,l2SaddleSwap,l2SaddleLpToken,l1Amb,l2Amb,l1FxBaseRootTunnel,l1PosRootChainManager,l1PosPredicate,l2MessengerProxy
0,('ethereum',",'USDC'",",'0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'",",'0x3666f603Cc164936C1b87e207F36BEBa4AC5f18a'",",12650032",",''",",''",",''",",''",",''",",''",",''",",''",",''",",''",",''",",''",",''",",''",",'')"
1,",('gnosis'",",'USDC'",",''",",''",",16617211",",'0x88ad09518695c6c3712AC10a214bE5109a655671'",",'0x12e59C59D282D2C00f3166915BED6DC2F5e2B5C7'",",'0xf6A78083ca3e2a662D6dd1703c939c8aCE2e268d'",",'0xDDAfbb505ad214D7b80b1f830fcCc89B60fb7A83'",",'0x25D8039bB044dC227f741a9e381CA4cEAE2E6aE8'",",'0x9ec9551d4A1a1593b0ee8124D98590CC71b3B09D'",",'0x76b22b8C1079A44F1211D867D68b1eda76a635A7'",",'0x5C32143C8B198F392d01f8446b754c181224ac26'",",'0x9D373d22FD091d7f9A6649EB067557cc12Fb1A0A'",",'0x4C36d2919e407f0Cc2Ee3c993ccF8ac26d9CE64e'",",'0x75Df5AF045d91108662D8080fD1FEFAd6aA0bb59'",",''",",''",",''",",'')"
2,",('polygon'",",'USDC'",",''",",''",",15810014",",'0xA0c68C638235ee32657e8f720a23ceC1bFc77C77'",",'0xD89ea85ee5dD2027dbC29Fbc198DC197D44c3d70'",",'0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'",",'0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'",",'0x25D8039bB044dC227f741a9e381CA4cEAE2E6aE8'",",'0x9ec9551d4A1a1593b0ee8124D98590CC71b3B09D'",",'0x76b22b8C1079A44F1211D867D68b1eda76a635A7'",",'0x5C32143C8B198F392d01f8446b754c181224ac26'",",'0x9D373d22FD091d7f9A6649EB067557cc12Fb1A0A'",",''",",''",",'0xD89ea85ee5dD2027dbC29Fbc198DC197D44c3d70'",",'0xA0c68C638235ee32657e8f720a23ceC1bFc77C77'",",'0x40ec5B33f54e0E8A33A975908C5BA1c14e5BbbDf'",",'0xbEC7b65bdD0e46aB5C7D37F99ECdb6F00B59667A')"
3,",('optimism'",",'USDC'",",''",",''",",1",",'0x0000000000000000000000000000000000000000'",",'0x1ba1f1368ecEB7bFcbdE20e1F803771b7B401F7d'",",'0x4200000000000000000000000000000000000010'",",'0x7F5c764cBc14f9669B88837ca1490cCa17c31607'",",'0xa81D244A1814468C734E5b4101F7b9c0c577a8fC'",",'0x25D8039bB044dC227f741a9e381CA4cEAE2E6aE8'",",'0x2ad09850b0CA4c7c1B33f5AcD6cBAbCaB5d6e796'",",'0x3c0FFAca566fCcfD9Cc95139FEF6CBA143795963'",",'0x2e17b8193566345a2Dd467183526dEdc42d2d5A8'",",''",",''",",''",",''",",''",",'')"
4,",('arbitrum'",",'USDC'",",''",",''",",440027",",'0x0000000000000000000000000000000000000000'",",'0x15281083cFB969d5643604E5e7079296D9eb0B92'",",'0x0000000000000000000000000000000000000000'",",'0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8'",",'0x0e0E3d2C5c292161999474247956EF542caBF8dd'",",'0x0ce6c85cF43553DE10FC56cecA0aef6Ff0DD444d'",",'0xe22D2beDb3Eca35E6397e0C6D62857094aA26F52'",",'0x10541b07d8Ad2647Dc6cD67abd4c03575dade261'",",'0xB67c014FA700E69681a673876eb8BAFAA36BFf71'",",''",",''",",''",",''",",''",",'')"
5,",('ethereum'",",'USDT'",",'0xdAC17F958D2ee523a2206206994597C13D831ec7'",",'0x3E4a3a4796d16c0Cd582C382691998f7c06420B6'",",12860139",",''",",''",",''",",''",",''",",''",",''",",''",",''",",''",",''",",''",",''",",''",",'')"
6,",('gnosis'",",'USDT'",",''",",''",",17155195",",'0x88ad09518695c6c3712AC10a214bE5109a655671'",",'0xC08055b634D43F2176d721E26A3428D3b7E7DdB5'",",'0xf6A78083ca3e2a662D6dd1703c939c8aCE2e268d'",",'0x4ECaBa5870353805a9F068101A40E0f32ed605C6'",",'0xFD5a186A7e8453Eb867A360526c5d987A00ACaC2'",",'0x91f8490eC27cbB1b2FaEdd29c2eC23011d7355FB'",",'0x49094a1B3463c4e2E82ca41b8e6A023bdd6E222f'",",'0x3Aa637D6853f1d9A9354FE4301Ab852A88b237e7'",",'0x5b10222f2Ada260AAf6C6fC274bd5810AF9d33c0'",",'0x4C36d2919e407f0Cc2Ee3c993ccF8ac26d9CE64e'",",'0x75Df5AF045d91108662D8080fD1FEFAd6aA0bb59'",",''",",''",",''",",'')"
7,",('polygon'",",'USDT'",",''",",''",",17058878",",'0xA0c68C638235ee32657e8f720a23ceC1bFc77C77'",",'0xFC047884dE7797A3D9cbA42FBc3b675388A110d5'",",'0xc2132D05D31c914a87C6611C10748AEb04B58e8F'",",'0xc2132D05D31c914a87C6611C10748AEb04B58e8F'",",'0x6c9a1ACF73bd85463A46B0AFc076FBdf602b690B'",",'0x9F93ACA246F457916E49Ec923B8ed099

In [26]:
#write sql file
bridges.to_csv('outputs/hop_to_sql.txt', sep='\t', index=False)
#write cols
text_file = open('outputs/hop_dune_column_names.txt', "w")
n = text_file.write(col_names)
text_file.close()